In [4]:
!pip -q install pyspark
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.master("local[*]").appName("Spark RDD").getOrCreate()
sc = spark.sparkContext
print("Spark session started")

Spark session started


In [10]:
!spark-submit --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.1
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 17.0.16
Branch HEAD
Compiled by user heartsavior on 2024-02-15T11:24:58Z
Revision fd86f85e181fc2dc0f50a096855acf83a6cc5d9c
Url https://github.com/apache/spark
Type --help for more information.


**Βήμα 1ο**

In [6]:
from google.colab import files
#from google.colab import drive
#drive.mount('/content/drive')

#path = '/content/drive/My Drive/Colab Notebooks/Big Data/HW1/flights_2000.csv' # Example path
uploaded = files.upload()            # pick your CSV
filename = next(iter(uploaded.keys()))
df = spark.read.csv(filename, header=True, inferSchema=True)
#df.show(10)

Saving flights_2000.csv to flights_2000.csv


**Βήμα 2ο**

In [7]:
rdd = df.rdd
pairs = rdd.map(lambda d: (d["ORIGIN_AIRPORT"], d["DEP_DELAY"]))  # (key, value)
filtered_pairs = pairs.filter(lambda p: p[1] is not None and p[1] >= 0)

**Βήμα 3ο**

In [8]:
# 3) Avg per key με combineByKey (safe pattern)
def create_comb(v):
    # αρχικοποιεί accumulator ανά key
    return (v, 1)  # (sum, count)

def merge_val(acc, v):
    # προσθέτει νέα τιμή σε υπάρχον accumulator
    return (acc[0] + v, acc[1] + 1)

def merge_comb(acc1, acc2):
    # συγχωνεύει accumulators από διαφορετικά partitions
    return (acc1[0] + acc2[0], acc1[1] + acc2[1])

avg_rdd = (filtered_pairs
           .combineByKey(create_comb, merge_val, merge_comb)
           .mapValues(lambda acc: acc[0] / acc[1]))

# 4) Αποτέλεσμα
print("Average delay per airport (RDD):", sorted(avg_rdd.collect()))

Average delay per airport (RDD): [('ATL', 13.170731707317072), ('BOS', 12.705357142857142), ('CLT', 14.0), ('DEN', 13.96078431372549), ('DFW', 17.36), ('IAD', 12.073684210526316), ('JFK', 14.571428571428571), ('LAS', 14.514563106796116), ('LAX', 11.08421052631579), ('MCO', 13.158415841584159), ('MIA', 13.144444444444444), ('ORD', 15.177083333333334), ('PHX', 13.48913043478261), ('SEA', 15.69), ('SFO', 11.195402298850574)]


**Βήμα 4ο**

In [9]:
top_10 = avg_rdd.sortBy(lambda x: x[1], ascending=False).take(10)

print("Top 10 airports with the biggest mean delay (descending order):")
for airport, delay in top_10:
    print(f"Airport: {airport}, Average Delay: {delay:.2f} minutes")

Top 10 airports with the biggest mean delay (descending order):
Airport: DFW, Average Delay: 17.36 minutes
Airport: SEA, Average Delay: 15.69 minutes
Airport: ORD, Average Delay: 15.18 minutes
Airport: JFK, Average Delay: 14.57 minutes
Airport: LAS, Average Delay: 14.51 minutes
Airport: CLT, Average Delay: 14.00 minutes
Airport: DEN, Average Delay: 13.96 minutes
Airport: PHX, Average Delay: 13.49 minutes
Airport: ATL, Average Delay: 13.17 minutes
Airport: MCO, Average Delay: 13.16 minutes


1η εκτέλεση: 27s
* 1ο action: 0.2s
---

2η εκτέλεση: 11s
* 1ο action: 0.4s

---

3η εκτέλεση: 9s
* 1ο action: 0.22s
---

4η εκτέλεση: 8s
* 1ο action: 0.28s
---

5η εκτέλεση: 11s
* 1ο action: 0.13s
---

Μέσος χρόνος 3 ενδιαμέσων: 11.99s
